In [ ]:
import cProfile
import copy
import io
import itertools
import pstats
from typing import Iterator
from fastfusion.frontend._set_parsing import InvertibleSet
from fastfusion.frontend.arch import Leaf, Memory
from fastfusion.frontend.workload.workload_spec import Tensor, RankVariable
import fastfusion.frontend.mapping as mapping_import
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.single_einsum_mapper import iterate_mappings_constraints


# # Example mapping node
# type: "temporal"
# rank: P
# # Choose one of the following cases
# # Case 1.a
# tile_shape: 3   # will make tile shapes with shape 3
# # Case 1.b
# tile_shape: null # will create a sympy symbol to represent tile shape and use that
# # Case 2.a
# factor: 3       # will make 3 as evenly shaped possible tiles
# # Case 2.b
# factor: null    # will create a sympy symbol to represent the factor, then same as 2.a
# # Case 3   (I'm only showing null from now on)
# tile_pattern:
#   stride: null
#   initial_shape: null  # This will create tile like this [0, 1, ..., initial_shape - 1], [initial_shape, ..., initial_shape + stride - 1], [initial_shape + stride, ..., initial_shape + 2*stride - 1], ...
# # Case 4
# tile_pattern:
#   stride: null
#   shape: null      # This will create tile like this [0, 1, ..., shape-1], [stride, stride+1, ..., stride + shape-1], [2*stride, 2*stride + 1, ..., 2*stride + shape - 1], ...
#         choices = list(integer_factorizations_to_n_parts(rank_size, len(loops)))

# Tile shape constraint: Applies to all tensor(s) in a storage node for which that tile shape is relevant
# Loop bound constraint: Only for spatial

spec = Specification.from_yaml(
    "architecture/four_level.arch.yaml",
    "workloads/mha_full_new.yaml",
    "workloads/mha_full_new.renames.yaml",
)

workload = spec.workload
renames = spec.renames

einsum_name = "K"
einsum = workload.einsums[einsum_name]
rank_variables = einsum.rank_variables
tensors = einsum.tensors
rank_variable_to_size = {r: 16 for r in rank_variables}

# If there are two back-to-back storages for the same tensor & the outer is
# optional, then it is invalid.
import time

t0 = time.time()
mappings_count = 0
n_mappings = 0

# pr = cProfile.Profile()
# pr.enable()

for i, (mapping, constraints) in enumerate(iterate_mappings_constraints(spec, "Q")):
    print(f"{i}: {mapping.compact_string()}")
    for c in constraints:
        print(c)

# pr.disable()
# s = io.StringIO()
# ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
# ps.print_stats(30)  # Print top 30 time-consuming functions
# print(s.getvalue())

# TODO: Check for ranks not in the mapping and put them at the bottom
# TODO: What if there are no loops? 
# TODO: Set _must_exist for all backing storage nodes
# TODO: Constraint attacher
# TODO: Deprecate processors and BaseSpecification
# TODO: Can't have tile size constraints on backing memory

ModuleNotFoundError: No module named 'fastfusion.frontend._set_parse'